# Resource Estimates for Time evolution

In [1]:
# Let's read in the Carbon example provided by Sandia
import numpy as np
from mec_sandia.vasp_utils import read_vasp

ase_cell = read_vasp(f"../vasp_data/C_POSCAR")
# Next we can get some system paramters
volume_ang = ase_cell.get_volume()
print("Volume = {} A^3".format(volume_ang))

# To compute rs parameter we need volume in Bohr
from ase.units import Bohr
volume_bohr = volume_ang / Bohr**3
# and the number of valence electrons
num_carbon = len(np.where(ase_cell.get_atomic_numbers() == 6)[0])
# There is 1 hydrogen atom in the cell. Is this just a proton?
num_elec = 1 + num_carbon * 4
from mec_sandia.vasp_utils import compute_wigner_seitz_radius
# Get the Wigner-Seitz radius
rs = compute_wigner_seitz_radius(volume_bohr, num_elec)
print("rs = {} bohr".format(rs))
print("eta = {} ".format(num_elec))

Volume = 255.27133939199965 A^3
rs = 0.9289611708818379 bohr
eta = 513 


In [57]:
from mec_sandia.ft_pw_resource_estimates import pw_qubitization_costs 

num_bits_momenta = 6 # Number of bits in each direction for momenta
eps_total = 1e-3 # Total allowable error
num_bits_nu = 6 # extra bits for nu 
num_bits_nuc = 6 # extra bits for (quantum?) nuclear positions 
num_nuclei = 1# len(ase_cell.get_atomic_numbers) # Number of (quantum?) nuclei
toff, qubit = pw_qubitization_costs(np=num_bits_momenta, eta=num_elec, Omega=volume_bohr, eps=eps_total, nMc=8, nbr=8, L=num_nuclei)
print(f"Toffolis = {toff:4.3e}, qubits = {qubit}")

Toffolis = 7.360e+14, qubits = 10965.0


In [62]:
# Deuterium
import numpy as np
from mec_sandia.vasp_utils import read_vasp

ase_cell = read_vasp(f"../vasp_data/D_POSCAR")
# Next we can get some system paramters
volume_ang = ase_cell.get_volume()
print("Volume = {} A^3".format(volume_ang))

# To compute rs parameter we need volume in Bohr
from ase.units import Bohr
volume_bohr = volume_ang / Bohr**3
# and the number of valence electrons
num_atoms = len(ase_cell.get_atomic_numbers())
# There is 1 hydrogen atom in the cell. Is this just a proton?
num_elec = 1 + num_atoms 
from mec_sandia.vasp_utils import compute_wigner_seitz_radius
# Get the Wigner-Seitz radius
rs = compute_wigner_seitz_radius(volume_bohr, num_elec)
print("rs = {} bohr".format(rs))
print("eta = {} ".format(num_elec))

Volume = 577.1514824733873 A^3
rs = 0.8130499641788754 bohr
eta = 1730 


In [64]:
from mec_sandia.ft_pw_resource_estimates import pw_qubitization_costs 

num_bits_momenta = 6 # Number of bits in each direction for momenta
eps_total = 1e-3 # Total allowable error
num_bits_nu = 6 # extra bits for nu 
num_bits_nuc = 6 # extra bits for (quantum?) nuclear positions 
num_nuclei = len(ase_cell.get_atomic_numbers()) # Number of (quantum?) nuclei
toff, qubit = pw_qubitization_costs(np=num_bits_momenta, eta=num_elec, Omega=volume_bohr, eps=eps_total, nMc=8, nbr=8, L=num_nuclei)
print(f"Toffolis = {toff:4.3e}, qubits = {qubit}")

TypeError: object of type 'method' has no len()